In [1]:
from pathlib import Path

In [2]:
import pandas as pd

In [3]:
# from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')

In [4]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
df = pd.read_csv('DisasterTweetsTrain.csv', usecols=['text', 'target'])
print(df.shape)
df.sample(3)

(7613, 2)


,text,target
4115,Gotta love #summer in #Calgary. #yyc #hailstorm #crazyweather http://t.co/xQbWnLBBIu,1
6207,when you burp and smoke comes out,0
6132,Lol '@j2bone: *cousin ' @Foxy__Siren: Coursing* '@WEYREY_gidi: Now they are causing Di Maria.. LOL'',0


In [7]:
# designed by me, does not remove stopwords
def excess_remover(para):
    """makes the following changes:
        replace newline with space
        replace email ids
        replace hypertext
        replace special characters
        hypertext
        extraspaces
        longwords over 14 chrs
        the stop-words remover part requires import nltk, nltk.download('stopwords')"""
    email_removed = re.sub(pattern='[^\s]+@[^\s]+ ', repl='', string=para)
    newline_removed = re.sub(pattern='\n', repl=' ', string=email_removed)
    spe_chr_removed = re.sub(pattern='[^a-zA-Z0-9\s]', repl='', string=newline_removed)
    hyper_removed = re.sub(pattern='htt\S+', repl='', string=spe_chr_removed)
    extra_spc_removed = re.sub(pattern='\s+', repl=' ', string=hyper_removed)
    long_wd_removed = re.sub(pattern='\S{14,}\s', repl='', string=extra_spc_removed)
    individual_words = long_wd_removed.lower().split()
    # stops = set(stopwords.words('english'))
    # meaningful_words = [w for w in individual_words if w not in stops]

    return ' '.join(individual_words)

In [8]:
excess_remover(df.loc[119, 'text'])

'320 ir icemoon aftershock djicemoon dubstep trapmusic dnb edm dance ices'

In [9]:
X = df['text']
y = df['target']
X_train = X.iloc[:7500]
y_train = y.iloc[:7500]
X_test = X.iloc[7500:]
y_test = y.iloc[7500:]

In [10]:
test_path = Path('C:/') / 'Users' / 'New' / 'DataScienceFolder' / 'DisasterTweetsTrain.csv'
df_test = pd.read_csv(test_path)#, usecols=['text'])
df_test.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1


**Using pipeline**




In [11]:
from sklearn.pipeline import make_pipeline

In [12]:
pipe_vect_log = make_pipeline(TfidfVectorizer(), LogisticRegression())

In [13]:
# Cleaning training data
X_train_cleaned = X_train.apply(lambda x: excess_remover(x))

In [14]:
pipe_vect_log.fit(X_train_cleaned, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [15]:
# Testing one record

In [16]:
excess_removed = pd.Series(excess_remover(X_test.iloc[1]))
pipe_vect_log.predict(excess_removed)

array([1], dtype=int64)

In [17]:
# for preserving the model
import pickle

In [19]:
# generated and moved to Django folder
# pickle.dump(pipe_vect_log, open('Model_Lr_Tweets2023.pkl', 'wb'))
# print('rfr done')